<a href="https://colab.research.google.com/github/JanMeow/GNN_StackedRF_3D_Spatial_DeepLearning/blob/main/Chpater_2_GNN_Guess_the_GeometricType_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy
!pip install pytorch
!pip install torch_geometric

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pytorch)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.5 MB/s eta 0:00:00


In [2]:
import torch
print(torch.__version__)

2.6.0+cu124


In [3]:
print(f"python {torch.version.cuda}")

python 12.4


In [4]:
torch.cuda.is_available()

False

#Transforming our graph into PYG graph

Although our data is in graph structure, it is different from the PyG format of graph we need for training.
So the first step would be to turn our graph into a PyG *graph*


1.   Turn node index into integers
2.   Use node.near property to connect the indexs



In [5]:
# Getting the data and data engineering functions
!git clone https://github.com/JanMeow/3d_graph_data_engineering.git
# Install the dependency
!mv 3d_graph_data_engineering data_engineering
!pip install numpy pandas ifcopenshell

Cloning into '3d_graph_data_engineering'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 129 (delta 77), reused 83 (delta 36), pack-reused 0 (from 0)
Receiving objects: 100% (129/129), 346.40 KiB | 3.89 MiB/s, done.
Resolving deltas: 100% (77/77), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 6.6 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
import ifcopenshell
import sys
sys.path.append('/content/data_engineering')
from data_engineering import collision as C, utils as U, compute_proxy as CP

In [7]:
from pathlib import Path
# lets use the test ifc_model
ifc_path = "/content/data_engineering/data/ifc/ML_train_data_0.ifc"

In [8]:
model = ifcopenshell.open(ifc_path)
root = model.by_type("IfcProject")[0]

In [9]:
from utils import  Graph
# Create a graph and establish BVH Tree=
graph = Graph.create(root)
graph.build_bvh()

Function ended, No more spatial child
Graph created


In [10]:
# Set seeds
import os
import random


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # Optional: enforce deterministic behavior (slower but more consistent)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

In [11]:
for node in graph.node_dict.values():
      if node.geom_info != None:
          node.near = [graph[guid] for guid in graph.bvh_query(node.geom_info["bbox"])
                        if guid != node.guid]

# Turn the data into features embedding for the GNN

In [12]:
#Get Intrisic features for all nodes
intrinsic_features = [CP.get_Intrinsic_features(graph, key) for key in graph.node_dict.keys()]
print("Finished Extracting Intrinsic Features")

Finished Extracting Intrinsic Features


In [13]:
#Example of an intrisic features
df1 = pd.DataFrame(intrinsic_features)
df1.head()

,AABB_X_Extent,OOBB_X_Extent,AABB_Y_Extent,OOBB_Y_Extent,AABB_Z_Extent,OOBB_Z_Extent,AABB_base_area,OOBB_Base_area,World_X_Extent,World_Y_Extent,...,Surface_area,Volume,Largest_face_area,Largest_face_normal_has_Z_vector,Number_of_vertices_in_base,Number_of_vertices,Number_of_faces,Number_of_XYZ_aligned_faces,Number_of_Z_aligned_faces,Number_of_faces_in_largest_face
0,4.69,4.651975,6.78,6.953317,0.04,0.040000,15.94,32.35,27.68,13.68,...,32.549824,0.659685,15.94,True,6,12,20,8,8,4
1,1.98,0.280000,3.70,0.202695,0.28,4.058783,0.80,0.06,27.68,13.68,...,3.953537,0.223608,1.12,False,4,8,12,4,4,2
2,3.44,4.458813,3.08,0.199449,0.28,0.280000,0.87,0.89,27.68,13.68,...,4.335963,0.244300,1.24,False,4,8,12,4,4,2
3,4.18,3.491121,6.53,6.070073,0.04,0.040000,15.59,21.19,27.68,13.68,...,31.833550,0.623736,15.59,True,5,10,16,6,6,3
4,0.57,0.280000,1.78,0.206350,0.28,1.831194,0.33,0.06,27.68,13.68,...,1.731608,0.092064,0.51,False,4,8,12,4,4,2


In [14]:
df1["Z_axis_aligned"] = df1["Z_axis_aligned"].apply(lambda x: 1 if x else 0)
df1["Largest_face_normal_has_Z_vector"] = df1["Largest_face_normal_has_Z_vector"].apply(lambda x: 1 if x else 0)
df1.head(2)

,AABB_X_Extent,OOBB_X_Extent,AABB_Y_Extent,OOBB_Y_Extent,AABB_Z_Extent,OOBB_Z_Extent,AABB_base_area,OOBB_Base_area,World_X_Extent,World_Y_Extent,...,Surface_area,Volume,Largest_face_area,Largest_face_normal_has_Z_vector,Number_of_vertices_in_base,Number_of_vertices,Number_of_faces,Number_of_XYZ_aligned_faces,Number_of_Z_aligned_faces,Number_of_faces_in_largest_face
0,4.69,4.651975,6.78,6.953317,0.04,0.040000,15.94,32.35,27.68,13.68,...,32.549824,0.659685,15.94,1,6,12,20,8,8,4
1,1.98,0.280000,3.70,0.202695,0.28,4.058783,0.80,0.06,27.68,13.68,...,3.953537,0.223608,1.12,0,4,8,12,4,4,2


In [15]:
x = []
for feature in intrinsic_features:
  x.append([v for v in feature.values()])
x = torch.tensor(x, dtype=torch.float)

In [16]:
x

tensor([[ 4.6900,  4.6520,  6.7800,  ...,  8.0000,  8.0000,  4.0000],
        [ 1.9800,  0.2800,  3.7000,  ...,  4.0000,  4.0000,  2.0000],
        [ 3.4400,  4.4588,  3.0800,  ...,  4.0000,  4.0000,  2.0000],
        ...,
        [ 0.0700,  0.0700,  5.1800,  ..., 42.0000, 10.0000,  8.0000],
        [ 0.2000,  0.2000,  0.2000,  ..., 12.0000,  4.0000,  2.0000],
        [ 0.2000,  0.2800,  3.6700,  ..., 10.0000,  4.0000,  2.0000]])

In [17]:
#Encode target labels
targets = [node.geom_type for node in graph.node_dict.values()]

In [18]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(targets)  # Now y is a NumPy array of integers
y = torch.tensor(y, dtype=torch.long)

#Feature Enrichment

* 🧭 z-difference | Vertical offset between center points of source and *target*
* 📏 Euclidean distance | Total 3D distance between source and target
* 🧮 Relative size ratio | Volume or height ratio of source vs. target node
* 📐 Angular offset | Angle between principal axes
* 📦 Intersection volume or overlap | How much AABB or OOBB intersect

In [19]:
# Create Different Edges (After a few trals , I found that multi-encode works better than one hot encode)
directions = ['upper', 'lower', 'left', 'right', 'near', 'hr', 'vr']
def encode_relation(relations):
    one_hot = [0] * len(directions)
    for relation in relations:
        if relation in directions:
            one_hot[directions.index(relation)] = 1
    return np.array(one_hot)

In [20]:
edge_index = []
edge_attr = []
for index, node in enumerate(graph.node_dict.values()):
  node.index = index

for node in graph.node_dict.values():
  direct_neighbours = CP.assign_neighbours(node, atol = 0.01, return_type = "guid")
  horizontal_relatives = CP.get_edge_attr_horizontal_relatives(graph, node, extent = 0.05)
  vertical_relatives = CP.get_edge_attr_get_vertical_relatives(graph, node, extent = 0.05)
  for near_node in node.near:
    edge_index.append([node.index, near_node.index])
    attr = CP.get_edge_attr_for_GNN(node, near_node)
    if near_node.guid not in direct_neighbours:
      edge_attr.append(np.hstack((attr,encode_relation(["near"]))))
    else:
      idx = direct_neighbours.index(near_node.guid)
      edge_attr.append(np.hstack((attr,encode_relation([directions[idx], "near"]))))
    for hr_guid, hr_attr in horizontal_relatives.items():
      hr =graph[hr_guid]
      edge_index.append([node.index, hr.index])
      edge_attr.append(np.hstack((hr_attr,encode_relation(["hr"]))))
    for vr_guid, vr_attr in vertical_relatives.items():
      vr =graph[vr_guid]
      edge_index.append([node.index, vr.index])
      edge_attr.append(np.hstack((vr_attr,encode_relation(["vr"]))))

In [22]:
from torch_geometric.data import Data

# Convert edge_index to tensor
edge_index_tensor = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
edge_attr_tensor = torch.tensor(edge_attr, dtype=torch.float)  # if needed later

# Create data — use different name if you want to preserve the list
data = Data(x=x, edge_index=edge_index_tensor, y=y)

In [23]:
print("type:", type(data.edge_index))
print("shape:", data.edge_index.shape)
print("dtype:", data.edge_index.dtype)

type: <class 'torch.Tensor'>
shape: torch.Size([2, 258018])
dtype: torch.int64


In [24]:
data.num_node_features

34

# Split the data into k-fold for training and testing

In [25]:
from sklearn.model_selection import train_test_split

# Step 1: Split the node indices
num_nodes = data.num_nodes
node_indices = np.arange(num_nodes)

# Optional: Stratified sampling (preserves class balance)
train_idx, test_idx = train_test_split(
    node_indices,
    test_size=0.2,
    stratify=data.y.numpy(),  # ensures same label distribution
    random_state=42
)

In [26]:
# Create train/test masks
data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.train_mask[train_idx] = True
data.test_mask[test_idx] = True

# Create a GNN

We first try a version where edge attribute is not used

In [27]:
if hasattr(data, "edge_attr"):
  data.edge_attr = None

In [28]:
print("edge_attr exists:", hasattr(data, "edge_attr"))
print("edge_attr value:", data.edge_attr)

edge_attr exists: True
edge_attr value: None


In [29]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv # GCN conv does not work with edge attr but we try without attr first

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [30]:
model = GCN(input_dim=data.num_node_features, hidden_dim = 32, num_classes=len(le.classes_))

#Train the model

In [ ]:
input_dim = data.num_node_features
hidden_dim = 32
num_classes = len(le.classes_)

model = GCN(input_dim, hidden_dim, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Training loop
model.train()
for epoch in range(500):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f}")


Epoch 000 | Loss: 10.9617
Epoch 050 | Loss: 1.7235


# Evaluate the model

In [ ]:
model.eval()
with torch.no_grad():
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = correct / data.test_mask.sum()
    print(f"\n✅ Test Accuracy: {acc:.4f}")

# Tuning the model

The model has only 67% accuracy at first but it shows that it is learning something.
In the Stacked RF two step classification apporach, even with just the intrisic features, the models could reach over 90% accuracy.
So first, lets see if we can up the performance by
1. Increasing the epochs
2. Increase the layers in the models

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Custom GCN block
class GCNBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = GCNConv(in_channels, out_channels)
        self.activation = nn.ReLU()

    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        x = self.activation(x)
        return x

In [ ]:
class DeepGCN(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes):
        super().__init__()

        self.layers = nn.ModuleList()
        dims = [input_dim] + hidden_dims

        for i in range(len(hidden_dims)):
            self.layers.append(GCNBlock(dims[i], dims[i + 1]))

        self.final_conv = GCNConv(dims[-1], num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        for layer in self.layers:
            x = layer(x, edge_index)
            x = F.dropout(x, p=0.5, training=self.training)

        x = self.final_conv(x, edge_index)
        return F.log_softmax(x, dim=1)

In [ ]:
input_dim = data.num_node_features
hidden_dim = [32, 64, 64] # this now takes a list
num_classes = len(le.classes_)

model = DeepGCN(input_dim, hidden_dim, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Training loop also increased the epoch
model.train()
for epoch in range(500):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f}")


In [ ]:
model.eval()
with torch.no_grad():
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = correct / data.test_mask.sum()
    print(f"\n✅ Test Accuracy: {acc:.4f}")

We now try
1. Another model architecture
2. Add Edge Attrbute
3. Enginer the Edge attribure (Previously was not used but it is just one hot encode, sparse vector are not beneficial to GNN learning)

#Edge Attribute

In [ ]:
data = Data(x=x, edge_index=edge_index_tensor, edge_attr=edge_attr_tensor, y=y)

In [ ]:
# Create train/test masks
data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.train_mask[train_idx] = True
data.test_mask[test_idx] = True

In [ ]:
from torch_geometric.nn import NNConv
class EdgeAwareGNN(nn.Module):
  def __init__(self, input_dim, hidden_dim, num_classes, edge_attr_dim):
      super().__init__()
      self.edge_mlp1 = nn.Sequential(
          nn.Linear(edge_attr_dim, 64),
          nn.ReLU(),
          nn.Linear(64, input_dim * hidden_dim)
      )

      self.conv1 = NNConv(input_dim, hidden_dim, nn=self.edge_mlp1, aggr='mean')

      self.edge_mlp2 = nn.Sequential(
          nn.Linear(edge_attr_dim, 64),
          nn.ReLU(),
          nn.Linear(64, hidden_dim * num_classes)
      )

      self.conv2 = NNConv(hidden_dim, num_classes, nn=self.edge_mlp2, aggr='mean')

  def forward(self, data):
      x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
      x = self.conv1(x, edge_index, edge_attr)
      x = F.relu(x)
      x = F.dropout(x, p=0.5, training=self.training)
      x = self.conv2(x, edge_index, edge_attr)
      return F.log_softmax(x, dim=1)


In [ ]:
model = EdgeAwareGNN(
    input_dim=data.num_node_features,
    hidden_dim= 32,
    num_classes=len(torch.unique(data.y)),
    edge_attr_dim=data.edge_attr.shape[1]
)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(500):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f}")


In [ ]:
model.eval()
with torch.no_grad():
    pred = model(data).argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = correct / data.test_mask.sum()
    print(f"\n✅ Test Accuracy: {acc:.4f}")

Why is DeepGCN outperforming EdgeAwareGNN? | Likely Because...

1. One-hot edge features aren't expressive enough
2. Edge semantics are inconsistent / noisy
3. NNConv model is too complex for 250 nodes compared to GCNBlock, too little data for a complex model which results in overfitting
4. GCN is simply a better match for your current data


# Graph Attention

Since the model is not improving much by adding complexity,
it could be because now the averaging mechnicsm in Conv does not fit the case where certain nodes need more attention than the others.
In our case, we can try graph attention system

In [ ]:
from torch_geometric.nn import TransformerConv

class TransformerBlock(nn.Module):
    def __init__(self, in_dim, out_dim, edge_attr_dim, heads=4, dropout=0.5):
        super().__init__()
        self.conv = TransformerConv(
            in_channels=in_dim,
            out_channels=out_dim,
            heads=heads,
            dropout=dropout,
            edge_dim=edge_attr_dim,
            concat=True  # True = concat heads, False = average
        )
        self.output_dim = out_dim * heads
        self.activation = nn.ReLU()

    def forward(self, x, edge_index, edge_attr):
        x = self.conv(x, edge_index, edge_attr)
        x = self.activation(x)
        return x

In [ ]:
class GraphTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes, edge_attr_dim, heads=4, dropout=0.5):
        super().__init__()
        self.layers = nn.ModuleList()
        dims = [input_dim] + hidden_dims

        for i in range(len(hidden_dims)):
            block = TransformerBlock(
                in_dim=dims[i],
                out_dim=dims[i + 1],
                edge_attr_dim=edge_attr_dim,
                heads=heads,
                dropout=dropout
            )
            self.layers.append(block)
            dims[i + 1] = block.output_dim

        self.final_conv = TransformerConv(
            in_channels=dims[-1],
            out_channels=num_classes,
            heads=1,
            concat=False,
            dropout=dropout,
            edge_dim=edge_attr_dim
        )

        self.dropout = dropout

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        for layer in self.layers:
            x = layer(x, edge_index, edge_attr)
            x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.final_conv(x, edge_index, edge_attr)
        return F.log_softmax(x, dim=1)

In [ ]:
input_dim = data.num_node_features
hidden_dims = [32, 64, 64]
num_classes = len(le.classes_)
edge_attr_dim = data.edge_attr.shape[1]

model = GraphTransformer(
    input_dim=input_dim,
    hidden_dims=hidden_dims,
    num_classes=num_classes,
    edge_attr_dim=edge_attr_dim,
    heads=4,
    dropout=0.5
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
model.train()
for epoch in range(600):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f}")

In [ ]:
model.eval()
with torch.no_grad():
    pred = model(data).argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = correct / data.test_mask.sum()
    print(f"\n✅ Test Accuracy: {acc:.4f}")

#🧠 Next Steps: Why Do GNNs Underperform Classical ML in our case?

The accuracy of GNN comes much lower than the 2 stacked classifaction apporach using ensemble learnings or even to simple RNN and logistic regressions. Below could be the following reasons and next steps to work on


### 🔹 1. Weak Graph Structure
- GNNs depend on meaningful edges.
- If the graph is noisy, overly dense, or doesn’t reflect true relationships, it can hurt performance.

### 🔹 2. Strong Node Features
- If intrinsic features already contain enough signal, classical models (RF, KNN) can perform very well without needing context from neighbors.

### 🔹 3. Model Complexity
- GNNs have many hyperparameters and are harder to tune.
- They're more prone to overfitting, especially on smaller datasets.

### 🔹 4. Over-smoothing
- In deep GNNs, nodes can become indistinguishable as features are averaged across neighbors.

### 🔹 5. Homophily Assumption Doesn’t Hold
- Many GNNs assume nodes of the same class are connected.
- In heterophilic graphs (e.g., walls connected to windows), this assumption breaks down.

### 🔹 6. Data Efficiency
- Classical models train faster and often require less data.
- GNNs need more data and regularization to perform well.

### 🔹 7. Graph Construction is Hard
- If the edges are hand-crafted or inconsistent, GNN performance suffers.
- Classical ML can directly use engineered features like cluster stats, variance, etc.

#Next Steps

1. Add Edge attributes to .near instead of simple one hot encode
2. Include horizonal and vertical relatives and add attributes to the edge